<a href="https://colab.research.google.com/github/LeoSousaJesus/semana_deep_learning/blob/main/Brasilia_mais_ti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Visão Computacional com OpenCV e MediaPipe**

# Esse código apresenta um sistema de detecção de rosto e análise de expressão facial para detectar se uma pessoa está com os olhos fechados por muito tempo e abrindo a boca (bocejo), indicando sonolência e possível perigo, caso esteja, tocar os sons de alerta para os dois casos.

# Distância Euclidiana (1D,2D,3D, n....)
# Média (cálculo do EAR)

In [ ]:
# Para rodar o programa execute o comando: python prova.py

# 1. Importação das bibliotecas

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pygame
import os
import sys

- **cv2:** OpenCV para processamento de imagens e manipulação de câmera.
- **mediapipe:** Para detecção de rosto e pontos visíveis.
- **numpy:** Para manipulações numéricas (ex., vetores e cálculos).
- **time:** Para controlar o tempo (ex., duração dos olhos fechados).
- **pygame:** Para tocar arquivos de áudio como som de alerta.
- **os e sys:** Para operações no sistema (não são usados ​​neste trecho).

# 2. Inicialize o sistema de som

In [ ]:
pygame.mixer.music.load("alarmenavio.mp3")  # Alarme da boca
som_olho = pygame.mixer.Sound("businadj.mp3")  # Alarme dos olhos
som_assobio = pygame.mixer.Sound("assobio.mp3")  # Som para rosto detectado

- **pygame.mixer.init():** Inicializa o sistema de som do Pygame.
- **pygame.mixer.music.load("sonscerrado.mp3"):** Carrega o arquivo de som que será reproduzido quando necessário.

# 3. Definindo pontos dos olhos e boca

In [ ]:
p_olho_esq = [385, 380, 387, 373, 362, 263]
p_olho_dir = [160, 144, 158, 153, 33, 133]
p_olhos = p_olho_esq + p_olho_dir
p_boca = [82, 87, 13, 14, 312, 317, 78, 308]

- Essas listas armazenam os índices dos pontos específicos da face que representam os olhos e a boca. Esses índices são usados ​​para calcular as métricas de abertura e fechamento dos olhos e boca.

# 4. Função EAR (Eye Aspect Ratio) Proporção do Olho

In [ ]:
def calculo_ear(face, p_olho_dir, p_olho_esq):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_esq = face[p_olho_esq, :]
        face_dir = face[p_olho_dir, :]

        ear_esq = (np.linalg.norm(face_esq[0] - face_esq[1]) + np.linalg.norm(face_esq[2] - face_esq[3])) / (2 * (np.linalg.norm(face_esq[4] - face_esq[5])))
        ear_dir = (np.linalg.norm(face_dir[0] - face_dir[1]) + np.linalg.norm(face_dir[2] - face_dir[3])) / (2 * (np.linalg.norm(face_dir[4] - face_dir[5])))
    except:
        ear_esq = 0.0
        ear_dir = 0.0
    media_ear = (ear_esq + ear_dir) / 2
    return media_ear

## A função calcula o Eye Aspect Ratio (EAR), que indica o grau de abertura dos olhos:

- Converte coordenadas dos pontos visíveis em matrizes.
- Seleciona os pontos dos olhos e calcula a razão entre as distâncias verticais e horizontais (indicando se os olhos estão abertos ou internos).
- Retorna a média dos EAR dos olhos direito e esquerdo.

# 5. Função MAR (Mouth Aspect Ratio) Proporção de aspecto da Boca


In [ ]:
def calculo_mar(face, p_boca):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_boca = face[p_boca, :]

        mar = (np.linalg.norm(face_boca[0] - face_boca[1]) + np.linalg.norm(face_boca[2] - face_boca[3]) + np.linalg.norm(face_boca[4] - face_boca[5])) / (2 * (np.linalg.norm(face_boca[6] - face_boca[7])))
    except:
        mar = 0.0
    return mar

## A função calcula o Mouth Aspect Ratio (MAR), diminuindo o grau de abertura da boca (ex., bocejo):

- Semelhante ao EAR, mas calcula a relação usando pontos da boca.

# 6. Limiares e controle de sono

In [ ]:
ear_limiar = 0.27
mar_limiar = 0.5
dormindo = False  # Flag para controle dos olhos fechados
aberto_boca = False  # Flag para controle da boca aberta
tempo_olhos_fechados = 0.0  # Tempo que os olhos ficaram fechados
tempo_boca_aberta = 0.0  # Tempo que a boca ficou aberta
contagem_piscadas = 0  # Contagem de piscadas

- Defina os valores de limites para os EAR e MAR, e uma variável dormindo para monitorar o estado de sono (1 para dormir e 0 para acordar).

# 7. Inicialização da câmera

In [ ]:
cap = cv2.VideoCapture(0)

- Ative a câmera para capturar imagens ao vivo.

# 8. Configuração do MediaPipe e variáveis ​​de estado e controle de exibição das mensagens

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Estado do som
som_boca_tocando = False
som_olho_tocando = False
som_assobio_tocando = False

# Variáveis para controle de exibição das mensagens
mensagem_boca = ""
mensagem_olhos = ""
tempo_inicio_mensagem_boca = None
tempo_inicio_mensagem_olhos = None
duracao_mensagem = 3  # Duração das mensagens em segundos

- Inicialize as ferramentas de desenho e detecção de malha facial do MediaPipe. Define som_tocando para controlar o estado do som e também a exbição das mensagens de alerta.

# 9. Configuração da janela da câmera

In [ ]:
window_width = 800
window_height = 600
cv2.namedWindow('Camera', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Camera', window_width, window_height)

- Defina uma janela de exibição da câmera com dimensões específicas.

In [ ]:
# Variáveis de tempo para o aviso de rosto
rosto_detectado = False
tempo_inicial_aviso = None



*  Definindo o aviso de rosto detectado




# 10. Loop de captura e processamento de imagem

In [ ]:
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as facemesh:
    while cap.isOpened():
        sucesso, frame = cap.read()
        if not sucesso:
            print('Ignorando o frame vazio da câmera.')
            continue

- Configure o FaceMesh MediaPipe para detectar pontos visíveis. O loop principal lê frames da câmera e continua enquanto ela estiver aberta.


# 11. Processamento de rosto detectado

In [ ]:
        comprimento, largura, _ = frame.shape
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        saida_facemesh = facemesh.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

- Converta uma imagem para RGB, processe como o FaceMesh e converta de volta para BGR para exibir na janela.

# 12. Controle de reprodução de som para o rosto detectado, onde o alarme aciona quando um rosto for detectado na tela

In [ ]:
        if saida_facemesh.multi_face_landmarks:
            rosto_detectado = True
            tempo_inicial_aviso = time.time()

- Verifique se algum rosto foi detectado. Caso positivo, toque o som (e mantenha tocando) enquanto o rosto estiver visível. Se o rosto não for detectado, o som para.

# 13. Desenho de pontos e cálculo EAR/MAR

In [ ]:
        for face_landmarks in saida_facemesh.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    face_landmarks,
                    mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 220), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 204, 0), thickness=1, circle_radius=1)
                )

                face = face_landmarks.landmark

                ear = calculo_ear(face, p_olho_dir, p_olho_esq)
                mar = calculo_mar(face, p_boca)
                cv2.rectangle(frame,(0,1),(290,140),(58,58,55),-1)

- Desenha pontos postais e calcula EAR e MAR, exibindo os valores no frame da câmera.


# 14. Lógica para estado de sono

In [ ]:
             # Verificação da condição da boca aberta
                if mar > mar_limiar:
                    if not aberto_boca:
                        tempo_inicial_boca_aberta = time.time()
                        aberto_boca = True
                    else:
                        tempo_boca_aberta = time.time() - tempo_inicial_boca_aberta
                    estado_boca = "aberta"
                else:
                    tempo_boca_aberta = 0.0
                    aberto_boca = False
                    estado_boca = "fechada"

                # Verificação da condição dos olhos fechados
                if ear < ear_limiar:
                    if not dormindo:
                        tempo_inicial_olhos_fechados = time.time()
                        dormindo = True
                    else:
                        tempo_olhos_fechados = time.time() - tempo_inicial_olhos_fechados
                    estado_olho = "fechados"
                else:
                    if dormindo:
                        contagem_piscadas += 1
                    tempo_olhos_fechados = 0.0
                    dormindo = False
                    estado_olho = "abertos"

- Verifica o tempo em que os olhos ficam fechados e a boca fica aberta, alertando na tela se o tempo ultrapassar os limiares. Adiciona a contagem das piscadas para toda vez que um rosto for detectado.

# 15. Exibição dos estados e tempos na tela

In [ ]:
      cv2.rectangle(frame, (0, 0), (320, 80), (58, 58, 55), -1)  # Fundo do quadro
      cv2.putText(frame, f"MAR Boca: {estado_boca} - {round(tempo_boca_aberta, 2)}s",
            (10, 30), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1)
      cv2.putText(frame, f"EAR Olhos: {estado_olho} - {round(tempo_olhos_fechados, 2)}s",
            (10, 60), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1)
      cv2.putText(frame, f"Piscadas: {contagem_piscadas}",
            (10, 90), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1)


- Exibe as informações de estados (aberto ou fechado) e tempos de duração dos olhos e boca na tela da câmera. desenhamos retângulos para o fundo e os textos são exibidos com informações sobre o estado da boca (aberta/fechada), tempo que a boca ficou aberta, estado dos olhos (abertos/fechados), tempo que os olhos ficaram fechados e a contagem de piscadas.

# 16. Alarme para boca aberta

In [ ]:
        if tempo_boca_aberta >= 1.5 and not som_boca_tocando:
            mensagem_boca = f"Alerta de sono, bocejando!"
            tempo_inicio_mensagem_boca = time.time()
            pygame.mixer.music.play(-1)
            som_boca_tocando = True
        elif tempo_boca_aberta == 0.0 and som_boca_tocando:
            pygame.mixer.music.stop()
            som_boca_tocando = False

- Verifica se a boca está aberta por mais de 1.5 segundos. Se estiver, toca um som de alarme e exibe uma mensagem na tela. Quando a boca é fechada, o som para.

# 17. Alarme para olhos fechados

In [ ]:
            if tempo_olhos_fechados >= 1.0 and not som_olho_tocando:
                mensagem_olhos = "Olhos fechados por muito tempo!"
                tempo_inicio_mensagem_olhos = time.time()
                som_olho.play()
                som_olho_tocando = True
            elif tempo_olhos_fechados == 0.0 and som_olho_tocando:
                som_olho.stop()
                som_olho_tocando = False


- Verifica se os olhos estão fechados por mais de 1 segundo. Se estiverem, toca um som de alarme e exibe uma mensagem na tela. Quando os olhos são abertos, o som para.

# 18. Exibir mensagens no canto inferior da tela

In [ ]:
              altura, largura, _ = frame.shape
              if mensagem_boca and (time.time() - tempo_inicio_mensagem_boca <= duracao_mensagem):
                  cv2.rectangle(frame, (0, altura - 70), (largura, altura), (0, 0, 0), -1)  # Fundo preto
                  cv2.putText(frame, mensagem_boca, (10, altura - 30), cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
              else:
                  mensagem_boca = ""  # Limpa a mensagem após a duração

              if mensagem_olhos and (time.time() - tempo_inicio_mensagem_olhos <= duracao_mensagem):
                  cv2.rectangle(frame, (0, altura - 140), (largura, altura - 70), (0, 0, 0), -1)  # Fundo preto
                  cv2.putText(frame, mensagem_olhos, (10, altura - 100), cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
              else:
                  mensagem_olhos = ""  # Limpa a mensagem após a duração


- Exibe mensagens sobre a condição da boca e olhos no canto inferior da tela. As mensagens são exibidas por um tempo determinado (duracao_mensagem) e depois são limpas.

# 19. Atualiza o estado e tempo quando nenhum rosto é detectado

In [ ]:
              if rosto_detectado:
                  contagem_piscadas = 0  # Reinicia a contagem de piscadas quando nenhum rosto é detectado
                  tempo_inicial_aviso = time.time()
              rosto_detectado = False

- reinicia a contagem de piscadas quando nenhum rosto é detectado e atualiza o tempo de início do aviso.

# 20. Exibe aviso de rosto detectado ou não detectado no canto superior direito

In [ ]:
              if tempo_inicial_aviso and (time.time() - tempo_inicial_aviso <= 5):
                  mensagem = "Rosto detectado" if rosto_detectado else "Nenhum rosto detectado"
                  cor_texto = (0, 255, 0) if rosto_detectado else (0, 0, 255)  # Verde para detectado, vermelho para não detectado
                  largura_frame = frame.shape[1]
                  tamanho_texto = cv2.getTextSize(mensagem, cv2.FONT_HERSHEY_DUPLEX, 1.0, 2)[0]  # Obtém o tamanho do texto
                  posicao_x = largura_frame - tamanho_texto[0] - 10  # Calcula a posição x do texto para o canto direito
                  posicao_y = 30  # Altura fixa no canto superior
                  cv2.putText(frame, mensagem, (posicao_x, posicao_y), cv2.FONT_HERSHEY_DUPLEX, 1.0, cor_texto, 2)

                  # Tocar som de assobio quando rosto detectado
                  if rosto_detectado and not som_assobio_tocando:
                      som_assobio.play()
                      som_assobio_tocando = True
              else:
                  # Para o som quando o tempo de exibição da mensagem acaba
                  if som_assobio_tocando:
                      som_assobio.stop()
                      som_assobio_tocando = False

- exibe um aviso no canto superior direito indicando se um rosto foi detectado ou não. Se um rosto é detectado, toca um som de assobio. Se não, para o som.

# 21. Exibir o frame da câmera e verificar a tecla de saída

In [ ]:
cv2.imshow('Camera', frame)
if cv2.waitKey(5) & 0xFF == 27:
    break

- exibe o frame da câmera e verifica se a tecla "ESC" foi pressionada para sair do loop.

# 22. Libera a câmera e fecha todas as janelas

In [ ]:
cap.release()
cv2.destroyAllWindows()

- libera a câmera e fecha todas as janelas abertas pelo OpenCV.